# Find Resonant Tip Frequency

In [1]:
import asyncio
import neaspec
import configuration as config

context = await neaspec.connect(config.db, config.vm, config.dll)

> Redirecting unknown loading assembly System.Runtime.CompilerServices.Unsafe, Version=4.0.4.1, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a
> Configuration in local file C:\Users\Patrick\AppData\Local\Python_Software_Foundatio\DefaultDomain_Path_mndturxegrmy5qhqcgfnwau3k5cajhfi\3.7.7\user.config
> Loaded from database group: Nea.Client.Hardware.Microscope.Properties.Settings
> Loaded from database group: Nea.Client.Hardware.Camera.Properties.Settings
> No camera found. Discovery result: Success
> Waiting for initialization...
> Trying to connect to 127.0.0.1
> Trying to connect to 127.0.0.1
> Trying to connect to 127.0.0.1
> Connected to 127.0.0.1
> Connected to 127.0.0.1
> Connected to 127.0.0.1
> Receive thread (Nea.Client.Hardware.MotorRpc) started.
> Receive thread (Nea.Client.Hardware.PreviewRpc) started.
> Client 1.10.7123 is incompatible with Server 1.10.7131.
> Client 1.10.7123 is incompatible with Server 1.10.7131.
> Receive thread (Nea.Client.Hardware.Rpc) started.
> Cli

When subscribing for receiving notifications e.g. `Logic.ResonantFrequencyFound` it is very important to unsubscribe. Therefore we can wrap the sweep in a class with Enter/Exit functions.

In [2]:
import Nea.Client.SharedDefinitions as shared

class Sweep:
	def onSweepSucceeded(self, o, e):
		print(f'Resonant Frequency Found: {e.ResonantFrequency} Hz')
		self.res_frequ = e.ResonantFrequency
		self.res_ampli = e.AmplitudeOfResonantFrequency
		self.res_phase = e.PhaseOfResonantFrequency
		self.isSweeping = False

	def onSweepFailed(self, o, e):
		print(f'Resonant Frequency Not Found, but Maximum at: {e.ResonantFrequency} Hz')
		self.isSweeping = False

	def __init__ (self, f_start, f_end):
		self.frequencyRange = shared.Range.FromOffsetAndLength(f_start, f_end - f_start)
		
        # Check if it currently possible to sweep (retracted out of contacted etc...)
		if context.Logic.FindResonantFrequency.CanExecute(self.frequencyRange):
			context.Logic.FindResonantFrequency.Execute(self.frequencyRange)
			print(f'Started to sweep in the range {self.frequencyRange.Low/1e3} ... {self.frequencyRange.High/1e3} kHz!')
			self.isSweeping = True
		else:
			print("Cannot sweep in the current state")
			self.isSweeping = False

	def __enter__ (self):            
        # Subscribe to events
		context.Logic.ResonantFrequencyFound += self.onSweepSucceeded
		context.Logic.FailedToFindResonantFrequency += self.onSweepFailed
		return self
	
	def __exit__(self, exception_type, exception_value, traceback):
        # Unsubscribe from events
		context.Logic.ResonantFrequencyFound -= self.onSweepSucceeded
		context.Logic.FailedToFindResonantFrequency -= self.onSweepFailed


In [3]:
from time import sleep

# sweep will start immediately and run in the background
with Sweep(260000, 270000) as sweep:

    # display the status
    while sweep.isSweeping:
        sleep(1)
        print(f'{context.Microscope.HeadFrequency}\t{context.Microscope.MechanicalAmplitude}') 


# after resonance successfully detected, 
# the resonance frequency is set automatically.
sleep(2)
print(f'Output after sweep: {context.Microscope.HeadFrequency} Hz')



Started to sweep in the range 260.0 ... 270.0 kHz!
260400.0	5.361146450042725
260800.0	4.982727527618408
261180.0	4.658880233764648
261560.0	4.36388635635376
261920.0	4.110353469848633
262360.0	3.8289902210235596
262740.0	3.6089887619018555
263100.0	3.417056083679199
263460.0	3.240084648132324
263840.0	3.066762685775757
264240.0	2.8996760845184326
264620.0	2.7541964054107666
265000.0	2.6188082695007324
265380.0	2.493584156036377
265760.0	2.3766677379608154
266120.0	2.273970127105713
266500.0	2.17165470123291
266860.0	2.0815823078155518
267240.0	1.9925411939620972
267640.0	1.9051493406295776
268060.0	1.819404125213623
268440.0	1.7470018863677979
268800.0	1.681541919708252
269200.0	1.6136181354522705
269600.0	1.5498039722442627
269960.0	1.4958199262619019
Resonant Frequency Not Found, but Maximum at: 260020.0 Hz
249455.1238328682	140.2034912109375
Output after sweep: 249455.1238328682 Hz
